# DATASET

In [4]:
#all imports
import pickle
import numpy as np
import pandas as pd

#import tensorflow_hub as hub

from tensorflow.keras.models import Model

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
import re
from tqdm import tqdm

from nltk.tokenize import word_tokenize 
from nltk import pos_tag 

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Conv2D, Flatten , Input , Conv1D , Concatenate , MaxPooling1D , Dropout , Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
import datetime

from keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from sklearn.metrics import  f1_score , roc_auc_score
import os

import tensorflow as tf
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import nltk.translate.bleu_score as bleu


tf.__version__


'2.8.2'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data = pd.read_csv('/content/drive/MyDrive/new_data.csv')
print(data.shape)
data.head(5)

(498362, 2)


,correct,incorrect
0,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .
1,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang - 8 was ..."
2,His Kanji ability is much better than mine .,His Kanji 's ability is much better than me .
3,"We 've known each other for only half a year ,...","We 've known each other for only half a year ,..."
4,I heard a sentence last night when I was watch...,I heard a sentence last night when I watched TV .


In [7]:
# ckeck if any missing value is present
data.isnull().values.any()

True

In [8]:
data.dropna(inplace=True)
data.shape

(498360, 2)

In [9]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True,drop=True)
data.shape

(496339, 2)

In [10]:
def clean(text):
    """
    takes string as input and
    removes characters inside (),{},[] and <>
    removes characters like -+@#^/|*(){}$~`
    we not not removing ,.!-:;"' as these characters are present in english language 
    """
    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)
    return text

data["correct"] = data["correct"].apply(clean)
data["incorrect"] = data["incorrect"].apply(clean)

In [11]:
def percentile(low,high,step,list_temp):
    """
    this function takes low, high, step size as input and prints percentiles accordingly
    """
    for i in np.arange(low,high,step):
        print(i,"percentile is ",np.percentile(list_temp ,i))

In [12]:
def sen_to_char(sen):
    return len([i for i in sen])

corr_length = data["correct"].apply(sen_to_char)
corr_length = list(corr_length)
len(corr_length)

496339

In [13]:
percentile(0,101,10,corr_length)
print("***************************************************************")
percentile(90,101,1,corr_length)

0 percentile is  0.0
10 percentile is  27.0
20 percentile is  35.0
30 percentile is  42.0
40 percentile is  48.0
50 percentile is  55.0
60 percentile is  63.0
70 percentile is  73.0
80 percentile is  86.0
90 percentile is  108.0
100 percentile is  2622.0
***************************************************************
90 percentile is  108.0
91 percentile is  112.0
92 percentile is  116.0
93 percentile is  120.0
94 percentile is  125.0
95 percentile is  131.0
96 percentile is  139.0
97 percentile is  149.0
98 percentile is  163.0
99 percentile is  188.0
100 percentile is  2622.0


In [14]:
# removing those data points which have correct sentence of length more than 108
index = []
for i in range(len(corr_length)):
    if corr_length[i] > 108:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(446785, 2)


In [15]:
incorr_length = data["incorrect"].apply(sen_to_char)
incorr_length = list(incorr_length)
len(incorr_length)

446785

In [16]:
percentile(0,101,10,incorr_length)
print("***********************************************************")
percentile(90,101,1,incorr_length)

0 percentile is  0.0
10 percentile is  25.0
20 percentile is  32.0
30 percentile is  38.0
40 percentile is  44.0
50 percentile is  50.0
60 percentile is  57.0
70 percentile is  64.0
80 percentile is  73.0
90 percentile is  85.0
100 percentile is  611.0
***********************************************************
90 percentile is  85.0
91 percentile is  87.0
92 percentile is  89.0
93 percentile is  91.0
94 percentile is  92.0
95 percentile is  95.0
96 percentile is  97.0
97 percentile is  99.0
98 percentile is  103.0
99 percentile is  107.0
100 percentile is  611.0


In [17]:
# removing those data points which have incorrect sentence of length more than 108
index = []
for i in range(len(incorr_length)):
    if incorr_length[i] > 108:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(443397, 2)


In [18]:
corr_length = data["correct"].str.split().apply(len)
corr_length = list(corr_length)

In [19]:
percentile(0,101,10,corr_length)
print("***************************************************************")
percentile(90,101,1,corr_length)

0 percentile is  0.0
10 percentile is  6.0
20 percentile is  8.0
30 percentile is  9.0
40 percentile is  10.0
50 percentile is  11.0
60 percentile is  13.0
70 percentile is  14.0
80 percentile is  16.0
90 percentile is  18.0
100 percentile is  47.0
***************************************************************
90 percentile is  18.0
91 percentile is  19.0
92 percentile is  19.0
93 percentile is  19.0
94 percentile is  20.0
95 percentile is  20.0
96 percentile is  21.0
97 percentile is  21.0
98 percentile is  22.0
99 percentile is  23.0
100 percentile is  47.0


In [20]:
# removing those data points which have correct sentence of length more than 24
index = []
for i in range(len(corr_length)):
    if corr_length[i] > 24:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(441931, 2)


In [21]:
incorr_length = data["incorrect"].str.split().apply(len)
incorr_length = list(incorr_length)

In [22]:
percentile(0,101,10,incorr_length)
print("***************************************************************")
percentile(90,101,1,incorr_length)

0 percentile is  0.0
10 percentile is  6.0
20 percentile is  7.0
30 percentile is  8.0
40 percentile is  10.0
50 percentile is  11.0
60 percentile is  12.0
70 percentile is  14.0
80 percentile is  15.0
90 percentile is  18.0
100 percentile is  31.0
***************************************************************
90 percentile is  18.0
91 percentile is  18.0
92 percentile is  18.0
93 percentile is  19.0
94 percentile is  19.0
95 percentile is  20.0
96 percentile is  20.0
97 percentile is  21.0
98 percentile is  21.0
99 percentile is  22.0
100 percentile is  31.0


In [23]:
# removing those data points which have incorrect sentence of length more than 25
index = []
for i in range(len(incorr_length)):
    if incorr_length[i] > 25:
        index.append(i)
        
data.drop(index,inplace=True)
data.reset_index(inplace=True,drop=True)
print(data.shape)

(441842, 2)


In [24]:
data["correct"] = data["correct"].astype(str)
data["incorrect"] = data["incorrect"].astype(str)

In [25]:
type(data.correct) , data.isnull().sum().sum()

(pandas.core.series.Series, 0)

In [26]:
length_max = max([len(i) for i in tqdm(data['correct'])])
length_min = min([len(i) for i in tqdm(data['correct'])])
avg = [len(i) for i in tqdm(data['correct'])]
length_avg = np.array([avg]).mean()

length_max , length_min , length_avg

100%|██████████| 441842/441842 [00:00<00:00, 2432180.87it/s]


(108, 0, 53.660100216819586)

In [27]:
CORRECT_SENTENCE_LEN = data['correct'].str.split().apply(len) 
ERRONEOUS_SENTENCE_LEN = data['incorrect'].str.split().apply(len)

In [28]:
for i in range(0,101,10):
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))
for i in range(90,101):
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))
for i in [99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100]:
    print(i,np.percentile(ERRONEOUS_SENTENCE_LEN, i))

0 0.0
10 6.0
20 7.0
30 8.0
40 10.0
50 11.0
60 12.0
70 14.0
80 15.0
90 18.0
100 25.0
90 18.0
91 18.0
92 18.0
93 19.0
94 19.0
95 20.0
96 20.0
97 21.0
98 21.0
99 22.0
100 25.0
99.1 22.0
99.2 23.0
99.3 23.0
99.4 23.0
99.5 23.0
99.6 23.0
99.7 24.0
99.8 24.0
99.9 24.0
100 25.0


SINCE 99.9% OF DATA HAS LENGTH LESS THAN 10 , SO SELECTING SENTENCE WITH WORD <10

In [29]:
data['CORRECT_SENTENCE_LEN'] = data['correct'].str.split().apply(len)
data = data[data['CORRECT_SENTENCE_LEN'] < 20]

data['ERRONEOUS_SENTENCE_LEN'] = data['incorrect'].str.split().apply(len)
data = data[data['ERRONEOUS_SENTENCE_LEN'] < 20]

#ADDING start and end IN THE SENTENCES
data['english_inp'] = '<start> ' + data['correct'].astype(str)
data['english_out'] = data['correct'].astype(str) + ' <end>'

data = data.drop(['correct','CORRECT_SENTENCE_LEN','ERRONEOUS_SENTENCE_LEN'], axis=1)
print(data.shape)
data.head()

(409715, 3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,incorrect,english_inp,english_out
0,And he took in my favorite subject like soccer .,<start> And he took in my favorite subjects li...,And he took in my favorite subjects like socce...
1,"Actually , who let me know about Lang was him .","<start> Actually , he was the one who let me k...","Actually , he was the one who let me know abou..."
2,His Kanji 's ability is much better than me .,<start> His Kanji ability is much better than ...,His Kanji ability is much better than mine . <...
4,I heard a sentence last night when I watched TV .,<start> I heard a sentence last night when I w...,I heard a sentence last night when I was watch...
5,"When you go downhill , you have to stick out y...","<start> When you go downhill , you have to sti...","When you go downhill , you have to stick out y..."


### Getting train and test 

In [30]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data, test_size=0.2)

In [31]:
print(train.shape, validation.shape)
#ADDING TO <end> TO ONE OF THE SENTENCES SO THAT TOKENIZER LEARNS THE WORD <end>
train.iloc[0]['english_inp']= str(train.iloc[0]['english_inp'])+' <end>'
train.iloc[0]['english_out']= str(train.iloc[0]['english_out'])+' <end>'

(327772, 3) (81943, 3)


In [32]:
max_encoder_length = max([len(i) for i in tqdm(train['incorrect'])])
max_decoder_length = max([len(i) for i in tqdm(train['english_inp'])])
print(max_encoder_length , max_decoder_length)

100%|██████████| 327772/327772 [00:00<00:00, 1710270.65it/s]


108 116


In [33]:
train.sample(2)

,incorrect,english_inp,english_out
426921,Could you teach me right ways ?,<start> Could you tell me a right way ?,Could you tell me a right way ? <end>
115732,there was Music fountain .,<start> There was a music fountain .,There was a music fountain . <end>


In [34]:
validation.sample(2)

,incorrect,english_inp,english_out
308611,They needs help .,<start> They need help .,They need help . <end>
388907,"I have the tennis match tomorrow ,","<start> I have a tennis match tomorrow ,","I have a tennis match tomorrow , <end>"


**TOKENIING**

In [35]:
tknizer_ERRONEOUS_SENTENCE = Tokenizer()
tknizer_ERRONEOUS_SENTENCE.fit_on_texts(train['incorrect'].values)
tknizer_CORRECT_SENTENCE = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_CORRECT_SENTENCE.fit_on_texts(train['english_inp'].values)

In [36]:
vocab_size_CORRECT_SENTENCE=len(tknizer_CORRECT_SENTENCE.word_index.keys())
print(vocab_size_CORRECT_SENTENCE)
vocab_size_ERRONEOUS_SENTENCE=len(tknizer_ERRONEOUS_SENTENCE.word_index.keys())
print(vocab_size_ERRONEOUS_SENTENCE)

45186
59023


In [37]:
tknizer_CORRECT_SENTENCE.word_index['<start>'], tknizer_CORRECT_SENTENCE.word_index['<end>']

(1, 23110)

TOKENIZER WITH ENGLISH WORDS

In [38]:
embeddings_index = dict()
f = open('/content/drive/MyDrive/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((vocab_size_CORRECT_SENTENCE+1, 100))
for word, i in tknizer_CORRECT_SENTENCE.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(45187, 100)

In [39]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, enc_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.enc_units= enc_units
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
    def build(self, input_shape):
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder", input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    def call(self, input_sentances, training=True):
        input_embedd                        = self.embedding(input_sentances)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    def get_states(self):
        return self.lstm_state_h,self.lstm_state_c


In [40]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, input_length, dec_units):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.input_length = input_length
    
    def build(self, input_shape):
        # we are using embedding_matrix weights and not training the embedding layer
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix],input_shape=(self.vocab_size,))
        self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM")
        
    def call(self, target_sentances, state_h, state_c):
        target_embedd           = self.embedding(target_sentances)
        lstm_output, _,_        = self.lstm(target_embedd, initial_state=[state_h, state_c])
        return lstm_output

## Data pipeline

In [41]:
class Dataset:
    def __init__(self, data, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, max_len):
        self.encoder_inps = data['incorrect'].values
        self.decoder_inps = data['english_inp'].values
        self.decoder_outs = data['english_out'].values
        self.tknizer_CORRECT_SENTENCE = tknizer_CORRECT_SENTENCE
        self.tknizer_ERRONEOUS_SENTENCE = tknizer_ERRONEOUS_SENTENCE
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ERRONEOUS_SENTENCE.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)




In [42]:
class Dataloder(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

## Model

In [43]:
class vanilla_model(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__() 
        self.encoder = Encoder(vocab_size=vocab_size_ERRONEOUS_SENTENCE + 1, embedding_dim=100, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=vocab_size_CORRECT_SENTENCE + 1, embedding_dim=100, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        
        
    def call(self, data):
        input,output = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input)
        decoder_output                       = self.decoder(output, encoder_h, encoder_c)
        output                               = self.dense(decoder_output)
        return output

In [44]:
train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)

train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=512)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 20) (512, 20) (512, 20)


In [45]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
vanilla = vanilla_model(encoder_inputs_length=16,decoder_inputs_length=16,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)
vanilla.compile(optimizer= optimizer, loss= loss_function, metrics=['accuracy'])

In [46]:
import datetime



In [ ]:
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
print(train_steps,valid_steps)

320 80


In [ ]:
vanilla  = vanilla_model(encoder_inputs_length=20,decoder_inputs_length=20,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
vanilla.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 50 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
vanilla.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=60 , validation_data=train_dataloader, validation_steps=valid_steps )#, callbacks=[stp, chkpt, tfboard]
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
vanilla.summary()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/60
320/320 [==============================] - 145s 431ms/step - loss: 3.2724 - val_loss: 2.9987
Epoch 2/60
320/320 [==============================] - 145s 454ms/step - loss: 2.8177 - val_loss: 2.6502
Epoch 3/60
320/320 [==============================] - 147s 460ms/step - loss: 2.5475 - val_loss: 2.4288
Epoch 4/60
320/320 [==============================] - 148s 461ms/step - loss: 2.3436 - val_loss: 2.2421
Epoch 5/60
320/320 [==============================] - 140s 436ms/step - loss: 2.1721 - val_loss: 2.0816
Epoch 6/60
320/320 [==============================] - 149s 465ms/step - loss: 2.0164 - val_loss: 1.9369
Epoch 7/60
320/320 [==============================] - 144s 449ms/step - loss: 1.8789 - val_loss: 1.7971
Epoch 8/60
320/320 [==============================] - 147s 458ms/step - loss: 1.7482 - val_loss: 1.6805
Epoch 9/60
320/320 [==============================] - 143s 446ms/step - loss: 1.6347 - val_loss: 1.5655
Epoch 10/60
320/320 [==============================] - 149s 467m

In [ ]:
os.mkdir('saved_model_newww')

In [ ]:
vanilla.save_weights('saved_model_newww/vanilla.h5')

In [ ]:
vanilla.load_weights("//content/drive/MyDrive/saved_model_newww/vanilla.h5")

In [ ]:
vanilla.summary()

Model: "vanilla_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  6280868   
                                                                 
 decoder_1 (Decoder)         multiple                  4902168   
                                                                 
 dense_1 (Dense)             multiple                  11658805  
                                                                 
Total params: 22,841,841
Trainable params: 22,841,841
Non-trainable params: 0
_________________________________________________________________


## Inference

In [ ]:
def inference(enc_inp,dec_inp):
        
    translation=""

    e_input=[]
    for i in enc_inp.split():
        if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

    #e_input = pad_sequences(e_input, maxlen=16, padding='post')


    e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

    #there is no onestep decoder in this thing, so I have to use the decoder input to predict output

    #decoder input
    d_input=[]
    for i in dec_inp.split():
        if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
            d_input.append(0)
        else:
            d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

    #d_input = pad_sequences(d_input, maxlen=16, padding='post')

    prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

    for word in prediction[0]:
        word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
        if word == "<end>":
            break
    translation += word + " "
    
    return translation
a = train.incorrect[4]
b =  train.english_inp[4]
pred = inference(a,b)
pred

'<end> '

In [ ]:
train.incorrect[1] , train.english_inp[1] , 

('Actually , who let me know about Lang   was him .',
 '<start> Actually , he was the one who let me know about Lang   . .')

In [ ]:
enc_inp = train.incorrect[1]
dec_inp =  train.english_inp[1] 

In [ ]:
translation=""

e_input=[]
for i in enc_inp.split():
    if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
        e_input.append(0)
    else:
        e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

#e_input = pad_sequences(e_input, maxlen=16, padding='post')


e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

#there is no onestep decoder in this thing, so I have to use the decoder input to predict output

#decoder input
d_input=[]
for i in dec_inp.split():
    if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
        d_input.append(0)
    else:
        d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

#d_input = pad_sequences(d_input, maxlen=16, padding='post')

prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

for word in prediction[0]:
    word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
    if word == "<end>":
        break
    translation += word + " "
translation

'who who who has happy reason who do me '

In [ ]:
sample_train = train.sample(1000)
sample_train

,incorrect,english_inp,english_out
434830,"Since I had no class in my university , I went...",<start> Even though I had no class in my unive...,"Even though I had no class in my university , ..."
65366,We should chose the fact .,<start> We should choose the fact .,We should choose the fact . <end>
112798,Massive earthquakes hit Japan while ago ! X,<start> Massive earthquakes hit Japan a while ...,Massive earthquakes hit Japan a while ago ! X ...
263823,Tired but feeling good .,<start> I 'm tired but feeling good .,I 'm tired but feeling good . <end>
321778,"this is four days , and this is going to bigin...","<start> It is four days long , and it is going...","It is four days long , and it is going to begi..."
...,...,...,...
329331,and he is characterized in this film as a grum...,<start> He is characterized in this film as a ...,He is characterized in this film as a grumpy m...
66596,It 's getting cooler after the typhoon has gone .,<start> It 's getting cooler now that the typh...,It 's getting cooler now that the typhoon has ...
266425,It is ours .,<start> It is ours,It is ours <end>
259369,"Every Saturday , my school holds football event .","<start> Every Saturday , my school holds a foo...","Every Saturday , my school holds a football ev..."


In [ ]:
def inference(enc_inp,dec_inp):
            
    translation=""

    e_input=[]
    for i in enc_inp.split():
        if tknizer_ERRONEOUS_SENTENCE.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(tknizer_ERRONEOUS_SENTENCE.word_index.get(i))

    #e_input = pad_sequences(e_input, maxlen=16, padding='post')


    e_output, e_hidden, e_cell = vanilla.layers[0](np.array([e_input], dtype='int32'))

    #there is no onestep decoder in this thing, so I have to use the decoder input to predict output

    #decoder input
    d_input=[]
    for i in dec_inp.split():
        if tknizer_CORRECT_SENTENCE.word_index.get(i) == None:
            d_input.append(0)
        else:
            d_input.append(tknizer_CORRECT_SENTENCE.word_index.get(i))

    #d_input = pad_sequences(d_input, maxlen=16, padding='post')

    prediction = vanilla.layers[2](vanilla.layers[1](np.array([d_input], dtype='int32'),e_hidden,e_cell))

    for word in prediction[0]:
        word = tknizer_CORRECT_SENTENCE.index_word[tf.argmax(word).numpy()]
        if word == "<end>":
            break
        translation += word + " "
    return translation
a = train.incorrect[5]
b =  train.english_inp[5]
pred = inference(a,b)
pred

'you you go downhill you you have to stick out your chest or go will go down '

In [ ]:
a = train.incorrect[10]
b =  train.english_inp[10]
pred = inference(a,b)
a  , b , pred

('The making souvenir is a hard and interesting work .',
 '<start> Making souvenirs is a hard but interesting work .',
 'making making is making hard and interesting and ')

In [ ]:
a = train.incorrect[12]
b =  train.english_inp[12]
pred = inference(a,b)
a  , b , pred

("That 's two balls : one with a cat inside , other with dog .",
 '<start> There are two balls : one with a cat inside , the other with dog .',
 'two two two balls with with with a cat inside with with other with him ')

In [ ]:
sample_train = train.sample(1000)
result = []
i=0
for i, j , k in tqdm(sample_train.values):
    # print(i , j , k)
    pred = inference(i , j)
    result.append(pred)

result

100%|██████████| 1000/1000 [01:19<00:00, 12.66it/s]


['is is not more than but it it it it is a castle named castle castle ',
 'was was was against ',
 'of of you know know know how to appreciate the taste of gourmet food ',
 'you you ever say to you ',
 "a a a 's team team won the game game game and the champions champion yesterday ",
 'helps helps helps us with communicating with foreigners ',
 'certainly certainly it certainly certainly as with the one one one ',
 'dentist dentist got rid of the bad part of the tooth ',
 'all all all boil boil soy beans ',
 'because those hostels have a community room ',
 'take take my dog and walk out for a walk on the beach ',
 'everyone everyone i i i hope hope wish time i the start i the the ',
 'and of getting up early ',
 'my my my junior boy shooting was there and and got a strong and and a big applause ',
 'if if you are not satisfied with my current job now now now ',
 'that that there a a a ban a smoking may as and could not be be just out ',
 "god god god 's title title title title for the 

In [ ]:
sample_train['PREDICTED_SENTENCE']= result
sample_train.head(20)

,incorrect,english_inp,english_out,PREDICTED_SENTENCE
94511,"It is not more famous than Kyoto , but it has ...","<start> It is no more famous than Kyoto , but ...","It is no more famous than Kyoto , but it has a...",is is not more than but it it it it is a castl...
370365,Seattle Mariners was against Losangels Angels .,<start> Seattle Mariners were went againstthe...,Seattle Mariners were went againsttheLos Ange...,was was was against
365572,"Because of you , I knew how to appreciate the ...","<start> Because of you , I knew how to appreci...","Because of you , I knew how to appreciate the ...",of of you know know know how to appreciate the...
140865,you ever to say with you self ?,"<start> Do you ever say to yourself ,","Do you ever say to yourself , <end>",you you ever say to you
76135,Japanese women soccer team won the World Cup g...,<start> The Japanese women 's soccer team won ...,The Japanese women 's soccer team won the Worl...,a a a 's team team won the game game game and ...
346202,The Internet helps us with communicatig with f...,<start> The Internet helps us with communicati...,The Internet helps us with communicating with ...,helps helps helps us with communicating with f...
371230,I thought it is certainly compared with the Sa...,<start> I thought it is certainly comparable w...,I thought it is certainly comparable with the ...,certainly certainly it certainly certainly as ...
318578,The dentist got rid out of the bad part on the...,<start> The dentist got rid of the bad part of...,The dentist got rid of the bad part of the too...,dentist dentist got rid of the bad part of the...
86311,"First of all , boil soy beens .","<start> First of all , boil soy beans .","First of all , boil soy beans . <end>",all all all boil boil soy beans
161455,"because , those hostels has a community room ,",<start> because those hostels have a community...,"because those hostels have a community room , ...",because those hostels have a community room


## BLEU Score

In [47]:
import nltk

In [ ]:
inp = sample_train.incorrect.values[20]
translate = sample_train.PREDICTED_SENTENCE.values[20]
print(inp)
print(translate)
print('BLEU score: {}'.format(nltk.translate.bleu_score.sentence_bleu(inp, translate)))

Some people always orders water  , but today nobody ordered it .
they they always gather water but but the nobody ordered it 
BLEU score: 0.7071067811865476


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
import nltk.translate.bleu_score as bleu

score = 0
for i in tqdm(range(1000)):
    inp = sample_train.incorrect.values[i]

    # print(inp)
    translate = sample_train.PREDICTED_SENTENCE.values[i]
    # print(translate)
    bleu = nltk.translate.bleu_score.sentence_bleu(inp, translate)
    score = score + bleu
average_bleu = score / 1000
print('the average BLEU score of these sentences. are :' , average_bleu)

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
100%|██████████| 1000/1000 [00:03<00:00, 277.59it/s]

the average BLEU score of these sentences. are : 0.737659553608704


In [ ]:
with open("encode.pickle","wb") as temp1:
   pickle.dump(vanilla,temp1)

INFO:tensorflow:Assets written to: ram://0b0016a4-8c4c-4b4c-8698-140533e2e359/assets


INFO:tensorflow:Assets written to: ram://0b0016a4-8c4c-4b4c-8698-140533e2e359/assets


# ATTENTION

In [48]:
#all imports
import pickle
import numpy as np
import pandas as pd

#import tensorflow_hub as hub
from tensorflow.keras.models import Model

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
import re
from tqdm import tqdm

from nltk.tokenize import word_tokenize 
from nltk import pos_tag 

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Conv2D, Flatten , Input , Conv1D , Concatenate , MaxPooling1D , Dropout , Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
import datetime

from keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from sklearn.metrics import  f1_score , roc_auc_score
import os

import tensorflow as tf
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import nltk.translate.bleu_score as bleu


tf.__version__


'2.8.2'

### <font color='blue'>**Implement custom encoder decoder and attention layers**</font>

<font color='blue'>**Encoder**</font>

In [49]:
class Encoder(tf.keras.Model):
    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        super().__init__()
        self.inp_vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        
        self.embedding = Embedding(input_dim=self.inp_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")


    def call(self,input_sequence,states):
        input_embedd = self.embedding(input_sequence)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd, states)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c
    
    def initialize_states(self,batch_size):
      self.lstm_state_h = tf.zeros([batch_size , self.lstm_size])
      self.lstm_state_c = tf.zeros([batch_size , self.lstm_size])
      return self.lstm_state_h,self.lstm_state_c


<font color='blue'>**Attention**</font>

In [50]:
class Attention(tf.keras.layers.Layer):

  def __init__(self,scoring_function, att_units):
    super(Attention, self).__init__()
    self.scoring_function = scoring_function
    self.att_units = att_units
    self.W1 = tf.keras.layers.Dense(att_units)
    self.W2 = tf.keras.layers.Dense(att_units)
    self.V = tf.keras.layers.Dense(1)

  
  def call(self,decoder_hidden_state,encoder_output):
    
    if self.scoring_function == 'dot':
        decoder_hidden_state_reshaped = tf.reshape(decoder_hidden_state , (decoder_hidden_state.shape[0],decoder_hidden_state.shape[1],1))

        #I WAS USING tf.keras.layers.Dot FOR DOT PRODUCT , BUT IT GAVE INCOMPATIBILITY IN SHAPES , SO NOW I VE USED tf.keras.layers.dot
        score =  tf.keras.layers.dot([ encoder_output , decoder_hidden_state_reshaped] , [2,1]) 
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    elif self.scoring_function == 'general':
        decoder_hidden_state_reshaped = tf.reshape(decoder_hidden_state , (decoder_hidden_state.shape[0],decoder_hidden_state.shape[1],1))
        W = tf.random.uniform(shape=[encoder_output.shape[0] , self.att_units , self.att_units])
        score =  tf.keras.layers.dot([ encoder_output , W] , [2,1]) 
        score =  tf.keras.layers.dot([ score , decoder_hidden_state_reshaped] , [2,1]) 
        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    elif self.scoring_function == 'concat':

        decoder_hidden_state_reshaped = tf.expand_dims(decoder_hidden_state, 1)
        score =  self.V(tf.nn.tanh(self.W1(decoder_hidden_state_reshaped) + self.W2(encoder_output)) )
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * encoder_output  #
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

        pass
    
    

<font color='blue'>**OneStepDecoder**</font>

In [51]:
class One_Step_Decoder(tf.keras.Model):
    def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(One_Step_Decoder, self).__init__()

        # Initialize decoder embedding layer, LSTM 
        self.tar_vocab_size = tar_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

        self.attention=Attention(score_fun,att_units)
        self.embedding = tf.keras.layers.Embedding(tar_vocab_size, embedding_dim)
        self.lstm = LSTM(self.dec_units , return_state=True, return_sequences=True, name="Decoder_LSTM")
        self.dense = tf.keras.layers.Dense(self.tar_vocab_size)

    def call(self,input_to_decoder, encoder_output, state_h,state_c):
        output = self.embedding(input_to_decoder) # (32, 1, 12)
        context_vector,attention_weights=self.attention(state_h,encoder_output)
        concat = tf.concat([tf.expand_dims(context_vector, 1), output], axis=-1)
        lstm_output, state_h, state_c = self.lstm(concat)
        
        output = self.dense(lstm_output)
        output = tf.reshape(output, (-1, output.shape[2]))

        return output,state_h,state_c,attention_weights,context_vector


<font color='blue'>**Decoder**</font>

In [52]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(Decoder , self).__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units

        self.onestep_decoder=One_Step_Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)

    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        all_outputs = tf.TensorArray(tf.float32, size=tf.shape(input_to_decoder)[1])
        for timestep in range(tf.shape(input_to_decoder)[1]):
            output,state_h,state_c,attention_weights,context_vector = self.onestep_decoder(input_to_decoder[: , timestep : timestep + 1] , \
                                                                                           encoder_output , decoder_hidden_state , decoder_cell_state)


            all_outputs = all_outputs.write(timestep , output)
        all_outputs = tf.transpose(all_outputs.stack() , [1,0,2])
        return all_outputs




In [53]:
out_vocab_size=13 
embedding_dim=12 
input_length=10
dec_units=16 
att_units=16
batch_size=32

target_sentences=tf.random.uniform(shape=(batch_size,input_length),maxval=10,minval=0,dtype=tf.int32)
encoder_output=tf.random.uniform(shape=[batch_size,input_length,dec_units])
state_h=tf.random.uniform(shape=[batch_size,dec_units])
state_c=tf.random.uniform(shape=[batch_size,dec_units])
score_fun = 'concat'
decoder=Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)
output=decoder(target_sentences,encoder_output, state_h, state_c)
print(output.shape)

(32, 10, 13)


<font color='blue'>**Encoder Decoder model**</font>

In [54]:
class Dataset:
    def __init__(self, data, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, max_len):
        self.encoder_inps = data['incorrect'].values
        self.decoder_inps = data['english_inp'].values
        self.decoder_outs = data['english_out'].values
        self.tknizer_CORRECT_SENTENCE = tknizer_CORRECT_SENTENCE
        self.tknizer_ERRONEOUS_SENTENCE = tknizer_ERRONEOUS_SENTENCE
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ERRONEOUS_SENTENCE.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_CORRECT_SENTENCE.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)


class Dataloder(tf.keras.utils.Sequence):
    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [55]:
train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 16)

train_dataset = Dataset(train, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)
test_dataset  = Dataset(validation, tknizer_ERRONEOUS_SENTENCE, tknizer_CORRECT_SENTENCE, 20)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=512)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 20) (512, 20) (512, 20)


In [56]:
class encoder_decoder(tf.keras.Model):
    def __init__(self,score_fun , encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__()
        #encoder decoder

        self.score_fun = score_fun

        self.encoder=Encoder(inp_vocab_size = vocab_size_ERRONEOUS_SENTENCE+1,embedding_size = 50,lstm_size = 64,input_length = encoder_inputs_length)
        self.decoder=Decoder(out_vocab_size = vocab_size_CORRECT_SENTENCE+1, embedding_dim = 100, input_length = decoder_inputs_length, dec_units =  64 \
                             ,score_fun = self.score_fun ,att_units = 64)

    def call(self,data):

        input,output = data[0], data[1]
        initial_state= self.encoder.initialize_states(batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input , initial_state)

        decoder_output= self.decoder(output,encoder_output, encoder_h, encoder_c)

        return decoder_output


In [60]:
batch_size=512
score_fun  = 'general'
att_units = 64
model_2  = encoder_decoder(score_fun = score_fun , encoder_inputs_length=20,decoder_inputs_length=10,output_vocab_size=vocab_size_CORRECT_SENTENCE)
optimizer = tf.keras.optimizers.Adam()
model_2.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#TRANING THE MODEL FOR 20 EPOCHS CAUSE , MORE TRAINING GIVES MORE RESULTS
log_dir="logs1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
model_2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=30, validation_data=train_dataloader, validation_steps=valid_steps , callbacks = checkpoint)
# model_1.fit_generator(train_dataloader,  epochs=4, validation_data=train_dataloader)
model_2.summary()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if sys.path[0] == '':


Epoch 1/30
320/320 [==============================] - 195s 595ms/step - loss: 4.2743 - val_loss: 3.9678
Epoch 2/30
320/320 [==============================] - 192s 599ms/step - loss: 3.9670 - val_loss: 3.9571
Epoch 3/30
320/320 [==============================] - 192s 599ms/step - loss: 3.8562 - val_loss: 3.7938
Epoch 4/30
320/320 [==============================] - 191s 596ms/step - loss: 3.8870 - val_loss: 3.8076
Epoch 5/30
320/320 [==============================] - 192s 599ms/step - loss: 3.7736 - val_loss: 3.9933
Epoch 6/30
320/320 [==============================] - 189s 592ms/step - loss: 3.8538 - val_loss: 3.8079
Epoch 7/30
320/320 [==============================] - 189s 590ms/step - loss: 3.8605 - val_loss: 3.7680
Epoch 8/30
320/320 [==============================] - 189s 591ms/step - loss: 3.8621 - val_loss: 3.9076
Epoch 9/30
320/320 [==============================] - 192s 598ms/step - loss: 3.8190 - val_loss: 3.8800
Epoch 10/30
320/320 [==============================] - 189s 590m

In [61]:
os.mkdir('/content/drive/MyDrive/saved_model_attention')

In [62]:
model_2.save_weights('/content/drive/MyDrive/saved_model_attention/attention.h5')

In [63]:
model_2.load_weights("/content/drive/MyDrive/saved_model_attention/attention.h5")
model_2.summary()

Model: "encoder_decoder_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  2980640   
                                                                 
 decoder_4 (Decoder)         multiple                  7514479   
                                                                 
Total params: 10,495,119
Trainable params: 10,495,119
Non-trainable params: 0
_________________________________________________________________


## <font color='blue'>**Inference**</font>

<font color='blue'>**CORRECTING THE INCORRECT SENTENCES**</font>

In [64]:
def predict(input_sentence):
    pred = []
    input_sequence = tknizer_CORRECT_SENTENCE.texts_to_sequences([input_sentence])
    result = ' '
    encoder_seq = pad_sequences(input_sequence, maxlen=20, dtype='int32', padding='post')  
    initial_state = model_2.layers[0].initialize_states(1)
    encoder_output, encoder_h, encoder_c = model_2.layers[0](tf.constant(encoder_seq), initial_state)
    start_index = tf.constant([[tknizer_CORRECT_SENTENCE.word_index['<start>']]])
    states = [encoder_h, encoder_c]

    for i in range(20): 
        # print(start_index)
        predicted_out,state_h,state_c,attention_weights,context_vector = model_2.layers[1].onestep_decoder(start_index, encoder_output , encoder_h, encoder_c )
        attention_weights = tf.reshape(attention_weights, (-1, ))
        start_index = np.reshape(np.argmax(predicted_out), (1, 1))
        pred.append(tknizer_CORRECT_SENTENCE.index_word[start_index[0][0]])
        if(pred[-1]=='<end>'):
            break
        translated_sentence = ' '.join(pred)
    return translated_sentence

inp = validation.values[2000][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

According to neighbors , the shop is very popular among people in Fukui .
i 'm a friend
BLEU score: 0.8857000285382948


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [65]:
inp = validation.values[2][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

Because they study hard foriegn lungage for the various goal .
i 'm a friend
BLEU score: 0.8857000285382948


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [66]:
inp = validation.values[20][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

Especially her listening comprehension skill was really good .
i 'm a friend
BLEU score: 0.8857000285382948


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [67]:
inp = validation.values[2000][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(bleu.sentence_bleu(inp, translate)))

According to neighbors , the shop is very popular among people in Fukui .
i 'm a friend
BLEU score: 0.8857000285382948


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [68]:
#USING THIS METHOD TO GET THE BLEU SCORE , THE REFERENCE NOTEBOOK METHOD SHOWS ME AN ERROR FOR 1000 FILES , BUT IT WORKS FINE FOR SINGLE FILE
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [69]:
inp = validation.values[2][0]
print(inp)
translate = predict(inp)
print(translate)
print('BLEU score: {}'.format(nltk.translate.bleu_score.sentence_bleu(inp, translate)))

Because they study hard foriegn lungage for the various goal .
i 'm a friend
BLEU score: 0.8857000285382948


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [70]:
sample_train = train.sample(1000)
result = []
i=0
for i, j , k in tqdm(sample_train.values):
    # print(i , j , k)
    pred = predict(i)
    result.append(pred)

result

100%|██████████| 1000/1000 [01:04<00:00, 15.58it/s]


["i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm really",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm going to visit",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm really",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm going to my friend",
 "i 'm a friend",
 "i 'm going to my friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm very good",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm going to visit",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm really",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm very much",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm going to my friend",
 "i 'm a friend",
 "i 'm a friend",
 "i 'm a

In [71]:
sample_train['PREDICTED_SENTENCE']= result
sample_train.head(20)

,incorrect,english_inp,english_out,PREDICTED_SENTENCE
243953,He said three students in the class including ...,"<start> He said three students in the class , ...","He said three students in the class , includin...",i 'm a friend
305521,I introduce my favourite things .,<start> I 'll now introduce my favourite things .,I 'll now introduce my favourite things . <end>,i 'm a friend
220752,"Last Sunday , I rented a movie called Patch A...","<start> Last Sunday , I rented a movie tited ...","Last Sunday , I rented a movie tited Patch Ad...",i 'm a friend
281708,"The other day , I went to the zoo .",<start> The other dayI went to the zoo .,The other dayI went to the zoo . <end>,i 'm a friend
166615,"After eating meals , we played games .","<start> After eating our meal , we played games .","After eating our meal , we played games . <end>",i 'm a friend
290983,What caught my eye were fluffy scarfs and whit...,<start> What caught my eye were fluffy scarfs ...,What caught my eye were fluffy scarfs and whit...,i 'm a friend
250168,The environment that I was raise was a living ...,<start> The environment that I was raised in w...,The environment that I was raised in was a liv...,i 'm really
317320,This picture is of sea at the island .,<start> This is a picture of the sea from the ...,This is a picture of the sea from the island ....,i 'm a friend
131786,But I did n't get anything ',<start> But I did n't get anything,But I did n't get anything <end>,i 'm a friend
366386,"Therefore if you do your best , you can do eve...","<start> Therefore if you do your best , you ca...","Therefore if you do your best , you can do eve...",i 'm going to visit


<font color='blue'>**Calculate BLEU score**</font>

In [72]:
#Create an object of your custom model.
#Compile and train your model on dot scoring function.
# Visualize few sentences randomly in Test data
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

#Sample example
import nltk.translate.bleu_score as bleu
reference = ['i am groot'.split(),] # the original
translation = 'it is ship'.split() # trasilated using model
print('BLEU score: {}'.format(bleu.sentence_bleu(reference, translation)))

BLEU score: 0
